<a href="https://colab.research.google.com/github/andrew-veriga/Tensorflow-labs/blob/master/Pushkin_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.
Это руководство скопировано с авторского с заменой обучающего набора данных на русский. В качестве датасета здесь используются стихи Пушкина.

Оригинальный Jupiter Notebook с примерами на шекспировских текстах:
https://www.tensorflow.org/tutorials/text/text_generation

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Генерация текста с RNN

В этом уроке показано, как создавать текст с помощью RNN на основе символов. Мы будем работать с набором данных пушкинских стихов по статье [«Необоснованная эффективность рекуррентных нейронных сетей»](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) Андрея Карпати.

 На основании последовательности символов из этих данных ('Пушки'), обучить модель предсказывать следующий символ в последовательности ('н'). Продолжение последовательности текста может быть сгенерировано повторным вызовом модели с входом, включающим последнюю предсказанную букву.

Примечание: Включите ускорение графического процессора, чтобы выполнить этот ноутбук быстрее.
In Colab: *Runtime > Change runtime type > Hardware acclerator > GPU*.

Этот руководство включает в себя запускаемый код, реализованный с помощью [tf.keras](https://www.tensorflow.org/programmers_guide/keras) и [eager execution](https://www.tensorflow.org/programmers_guide/eager). 
Ниже приводится образец вывода, когда модель в этом учебнике была тренирована в течение 100 эпох, и начала со строки 'И вот':

```
И вот мудрецов,
Глазами ученик,
И недоветельных и строгих),
Ученый малый, но Евгений
Наедине с своей душой
Был недоволен сам собой.
И поделом: в разборе строгом,
На тайники друг свободу заицуда
И шум немирных челноков.
Я вдаль уплыл, надежды полны;
```

Хотя некоторые предложения - грамматические, большинство из них не имеет смысла. Модель не понимает значения слов, но обратите внимание:

- Модель основана на последовательностях букв. Когда обучение началось, модель не знала, ни как пишется слово, ни даже то, что слово - это единица текста.

- Как показано ниже, модель обучается на небольших пакетах текста (по 100 символов каждый) и тем не менее способна генерировать более длинную последовательность текста с согласованной структурой.



## Setup

### Импорт TensorFlow and необходимых библиотек

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

###Загрузка текста из Google Drive

Чтобы файл был доступен из вашего диска, можно предоставить на него доступ на просмотр "для всех"

In [ ]:
# Импорт PyDrive и связанных с ним библиотек.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#https://drive.google.com/file/d/1pLbtXYa7kYSOoqgv3U_FPYyAvOrH0l0B/view?usp=sharing
# Download a file based on its file ID.
file_id = '1pLbtXYa7kYSOoqgv3U_FPYyAvOrH0l0B'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

### Read the data

First, look in the text:

In [ ]:
# Read, then decode for py2 compat.
text = downloaded.GetContentString()
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

In [ ]:
# Take a look at the first 200 characters in text
print(text[:200])

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

## Обработка текста

### Векторизация текста
Перед тренировкой нужно представить строки в виде чисел. Созадим две таблицы: одна для поиска числа по символу, другая - символа по числу.


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Теперь у нас есть числовое представление для каждого символа. Обратите внимание, что мы отображаем символ как индекс от 0 до
`len(unique)`.

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [ ]:
# Посмотрим, как первые 13 символов из текста отображаются в челые числа
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

### Прогнозирование

Задача, которой мы обучаем модель: **по заданному символу, или последовательности символов, выбрать  следующий символ.**

Поскольку RNN сохраняет внутреннее состояние, зависящее от ранее виденных элементов, то зная все символы, вычисленные до этого момента, она может предсказать следующий  символ на каждом шаге времени.


### Создание обучаемых примеров и меток
Теперь делим текст на примеры последовательностей. Каждая входящая последовательность будет содержать `seq_length` символов из текста.

Для каждой входящей последовательности соответствующие метки содержат такую же длину текста, но  смещены на один символ вправо.

Итак разобьем текст на куски `seq_length` + 1. Например, скажем, `seq_length=4`, и наш текст 'рыбка'. Входная последовательность будет 'рыбк', и целевая последовательность - 'ыбка'.

Для этого сначала используйте функцию 'tf.data.Dataset.from_tensor_slices' для преобразования вектора текста в поток индексов символов.


In [ ]:
# Максимальная длина предложения для единичного ввода символов
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Создаем обучающие примеры и метки
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Метод `batch` позволяет легко преобразовывать эти отдельные символы в последовательности требуемого размера.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

Каждую последовательность дублировать и сдвинуть для создания ввода и метки. Применим к каждому пакету метод `map` с простой функцией сдвига:

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Напечатаем первый пример и метку:

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Каждый индекс этих векторов обрабатывается как один шаг времени. Для ввода в шаге времени 0 модель получает индекс для первого символа 'Н' и пытается предсказать индекс для 'е' в качестве следующего символа. В следующий шаг времени она делает то же самое, но RNN рассматривает предыдущий контекст шага в дополнение к текущему символу ввода.

Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

### Создание тренировочных пакетов

Мы использовали `tf.data`, чтобы разделить текст на управляемые последовательности. Но прежде чем залить эти данные в модель, нам нужно перетасовать данные и упаковать их в пакеты.

In [ ]:
# Размер пакета
BATCH_SIZE = 64
# Размер буфера для перетасовки набора данных
# (данные TF предназначены для работы с возможными бесконечными последовательностями,
# Так что он не пытается перетасовать всю последовательность в памяти. Вместо этого
# он поддерживает буфер, в котором перемешивает элементы).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

## Строим модель

Для определения модели используйте tf.keras.Sequential. В этом простом примере в модели используются три слоя:

* `tf.keras.layers.Embedding`: Входиной слой. Таблица обучаемых поисков, которая свяжет  номер каждого символа с вектором размерностью `embedding_dim`;

* `tf.keras.layers.GRU`: Тип RNN с размером `units=rnn_units` (Тут также можно использовать слой LSTM.)

* `tf.keras.layers.Dense`: выходной слой с количеством юнитов `vocab_size` 


In [ ]:
# размер словаря
vocab_size = len(vocab)

# размерность векторов встраивания
embedding_dim = 256

# количество юнитов RNN 
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

Для каждого символа модель смотрит на эмбеддинг-вектор, запускает GRU на один шаг времени со эмбеддингом в качестве входа и выводит в полносвязный слой для генерации логистических юнитов, предсказывающих максимальное правдоподобие входа следующего символа:

![A drawing of the data passing through the model](https://github.com/andrew-veriga/Tensorflow-labs/blob/master/text_generation_training.png?raw=1)

## Пробуем модель
Теперь запустите модель, чтобы увидеть, что она ведет себя как ожидалось.

Сначала проверьте форму вывода:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

В приведенном выше примере длина ввода последовательности равна `100`, но модель может быть запущена на входных данных любой длины:


In [ ]:
model.summary()

Чтобы получить от модели предсказания, нам нужно выбрать из распределения на выходе актуальные индексы. Это распределение определяется логитами по словарю символов.

Примечание: Важно, что *выборка* из распределения, взятая просто как *argmax* распределения может легко привести модель к зацикливанию.

Попробуйте это для первого примера в пакете:


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

Это дает нам на каждом шаге во времени предсказание следующего индекса символов:

In [ ]:
sampled_indices

декодируем их, чтобы посмотреть текст, предсказанный этой нетренированной моделью:

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

## Тренировка модели

На этом этапе проблема может рассматриваться как стандартная задача классификации: По предыдущему состоянию RNN и входу на текущий шаг времени предсказать класс следующего символа.

### добавляем оптимизатор и функцию потерь

Стандартная функция потерь `tf.keras.losses.sparse_categorical_crossentropy` работает в этом случае, поскольку она применяется на самой последней размерности прогнозов.

Поскольку наша модель возвращает логиты, нам необходимо установить флаг `from_logits`


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Компилируем процедуру обучения с использованием метода `tf.keras.Model.compile`
Используем `tf.keras.optimizers.Adam` с аргументами по умолчанию и нашу функцию потерь.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Настройка контрольных точек

Используем `tf.keras.callbacks.ModelCheckpoint` чтобы обеспечить сохранение контрольных точек во время обучения:

In [ ]:
# Каталог для сохранения контрольных точек
checkpoint_dir = './training_checkpoints'
# Имена файлов контрольных точек
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Тренируем модель

In [ ]:
EPOCHS=10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

##Генерация текста
### Восстановим последнюю контрольную точку

для простоты, на этом шаге прогнозирования используем размер пакета 1.

Способ, которым состояние RNN передается от одного временного шага к другому, позволяет модели получать фиксированный размер пакета после построения.

Чтобы запустить модель с другим batch_size, нам нужно перестроить модель и восстановить веса с контрольной точки.


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

### Цикл генерации
Следующий блок кода генерирует текст:

* начинается с выбора стартовой строки, инициализации состояния RNN и задания количества символов для генерации.

* Получаем распределение предсказания следующего символа с помощью стартовой строки и состояния RNN.

* используем распределение по категориям для расчета индекса прогнозируемого символа. Этот прогнозируемый символ используется в качестве следующего входа в модель.

Состояние RNN, возвращенное моделью, снова отправяется в модель, так что теперь оно имеет больше контекста, а не только одно слово. После предсказания следующего слова, измененные состояния RNN снова подается обратно в модель, которая, пока обучается,  получает все больше контекста из ранее предсказанных слов.


![To generate text the model's output is fed back to the input](https://github.com/andrew-veriga/Tensorflow-labs/blob/master/text_generation_sampling.png?raw=1)

Посмотрев на сгенерированный текст, вы увидите, что модель знает, когда ставить большие буквы, делать абзацы и имитирует Пушкинский стиль. С небольшим количеством эпох обучения она еще не научилась формировать последовательные предложения.


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"и вот"))

Самое простое, что вы можете сделать для улучшения результатов, это тренировать его дольше (попробуйте `EPOCHS=30`).

Вы также можете поэкспериментировать с другой начальной строкой или попробовать добавить еще один слой RNN, чтобы повысить точность модели, или настроить параметр температуры, чтобы генерировать более или менее случайные прогнозы.

## Дополнение: индивидуальная настройка обучения

Описанная выше процедура обучения проста, но не дает вам особого контроля.

Итак, теперь, когда вы увидели, как запустить модель вручную, давайте распакуем цикл обучения и реализуем его самостоятельно. Это дает отправную точку, например, для реализации _curriculum learning_, чтобы помочь стабилизировать выходные данные модели без обратной связи.

Будем использовать `tf.GradientTape` для отслеживания градиентов. Вы можете почитать об этом здесь: [eager execution guide](https://www.tensorflow.org/guide/eager).

Процедура работает следующим образом:

* Во-первых, инициализируйте состояние RNN. Мы делаем это, вызывая метод `tf.keras.Model.reset_states`.

* Затем выполняется итерация по датасету (одна итерация для пакета) и вычисляется *прогноз* для пакета.

* Открывается объект `tf.GradientTape` и рассчитывается потеря в этом контексте.

* Вычисляются градиенты потерь по переменным модели, с использованием метода `tf.GradientTape.grads`.

* Наконец, следующий шаг обновления весов по вычисленным градиентам, с использованием метода оптимизатора `tf.train.Optimizer.apply_gradients`.


In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

In [ ]:
checkpoint_prefix.format(epoch=epoch)